In [1]:
import ray
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import xgboost
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from os.path import exists
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import datetime
import numpy as np
import time
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
dataset='Fault_data'
data_level="L1/"
path="fault_data/" + data_level

x_train = np.load(path+"X_train.npy")
x_test = np.load(path+"X_test.npy")
y_train = np.load(path+"y_train.npy")
y_test = np.load(path+"y_test.npy")

X=np.concatenate((x_train,x_test))

y=np.concatenate((y_train,y_test))

print("Shape of Train Data : {}".format(X.shape))
print("Shape of Test Data : {}".format(y.shape))

input_dim=X.shape[1]
num_classes=len(np.unique(y))

num_epochs=20
batch_size=32
num_P=2 #num_processors per one function

now = datetime.datetime.now()
Time_Now= now.strftime("%Y-%m-%d %H:%M:%S")
num_classes

Shape of Train Data : (33693, 15)
Shape of Test Data : (33693, 1)


7

In [3]:
cluster= True
PC=''
if (cluster):
    ray.init(address='auto') # parallel in cluster of PCs
    PC='Cluster'
else:
    ray.init() # parallel in one PC
    PC='Leovo'



2023-01-07 23:34:37,053	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 192.168.1.2:6379...
2023-01-07 23:34:37,061	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [4]:
#produce results
def write_results(results, path):
    
    file_exists = exists(path)
    if not(file_exists):
        with open(path,"a+") as f:
            f.write(results)
            f.write("\n")
    else:
        with open(path,"a+") as f:
            f.write(results)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20)

input_dim=x_train.shape[1]

def create_model():
    
    #early_stop = EarlyStopping(monitor='loss', patience=2)
    model = Sequential()

    model.add(Dense(16, activation='relu', input_shape=(input_dim,),kernel_initializer='random_uniform'))
    model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
    model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
    model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    
    return model

2023-01-07 23:34:37.246264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 23:34:37.977033: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-07 23:34:37.977061: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-07 23:34:39.652772: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
@ray.remote(num_cpus=num_P)
def run_model(i, X,x_train , y_train, x_test , y_test):
    
    t_temp=time.time()
    #individual classifier
    input_dim=x_train.shape[1]

    model1=create_model()
    #hist = model1.fit(x_train , y_train , epochs=20)
    #hist = model1.fit(x_train , y_train , epochs=20, batch_size=5, verbose=0)
    print(x_train.shape)
    hist = model1.fit(x_train , y_train , epochs=num_epochs, batch_size=batch_size, verbose=0)
    y_predict=model1.predict(x_test)
    y_predict=np.argmax(y_predict,axis=1)

    #_, accuracy = model1.evaluate(x_test, y_test)
    #Accuracy
    acc=accuracy_score(y_test, y_predict)
    prc=precision_score(y_test, y_predict, average='weighted')
    rec=recall_score(y_test, y_predict, average='weighted')
    f1=f1_score(y_test, y_predict, average='weighted')
    
 
    results = 'DL:%d' %(i) + ', Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
    results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', T:%.2f ' % (time.time()-t_temp) + ', PC: ' + PC
    results += ', Num Processors: ' + str(num_P) + ', Eb:%d ' %(num_epochs) + ', Bs:%d ' %(batch_size) + ', Date:' + Time_Now + '\n'
    
    predictions=model1.predict(X)
    max_indices = np.argmax(predictions,axis=1)

    return_values=[]
    return_values.append(np.array(max_indices))
    return_values.append(results)

    return return_values

t1=time.time()

Return_values=[]

Return_values=[run_model.remote(i, X,x_train , y_train, x_test , y_test) for i in range(5)]

#collect the output from classifiers that used original features

outputs_original=ray.get(Return_values)

all_time_before=time.time()-t1

#0 indicates to the the outputs_original
#1 indicates to results_original
predictions=outputs_original[0][0]
for i in range(1,5):
    predictions=np.vstack((predictions,outputs_original[i][0]))

predictions= np.transpose(predictions)

# concat predictions with olriginal features
new_X=np.concatenate((X, predictions), axis=1)
#new_X=predictions #np.concatenate((X, predictions), axis=1)
#====z_score normalization
scaler = StandardScaler()
scaler.fit(new_X)
new_X = scaler.fit_transform(new_X)
print(new_X.shape, predictions.shape)

new_x_train,new_x_test,y_train,y_test = train_test_split(new_X,y,test_size = 0.10)

input_dim=new_x_train.shape[1]


#Save the performance results of original classifiers
path2= path +"/results/Res_Singl_Parallel_DL_Original.txt" # the name of the file
results_before='Dataset:' + dataset + '=============================================\n'

for i in range(5):
    results_before+=outputs_original[i][1]

results_before+= 'Done...in time:' + str(round(all_time_before)) + '\n'
    
write_results(results_before, path2)



(pid=40636) 2023-01-07 23:34:44.187296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=40636) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=40636) 2023-01-07 23:34:44.407472: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=40636) 2023-01-07 23:34:44.407512: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=40637) 2023-01-07 23:34:44.157445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the followi

(run_model pid=40636) (449324, 15)
(run_model pid=40637) (449324, 15)


(pid=10497, ip=192.168.0.12) 2023-01-07 23:34:51.367915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=10497, ip=192.168.0.12) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=10462, ip=192.168.0.12) 2023-01-07 23:34:54.678854: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=10462, ip=192.168.0.12) 2023-01-07 23:34:54.678950: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=10497, ip=192.168.0.12) 2023-01-07 23:34:54.678854: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

(run_model pid=10462, ip=192.168.0.12) (449324, 15)
(run_model pid=10497, ip=192.168.0.12) (449324, 15)


(run_model pid=10462, ip=192.168.0.12) 2023-01-07 23:35:19.855372: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 26959440 exceeds 10% of free system memory.
(run_model pid=10497, ip=192.168.0.12) 2023-01-07 23:35:19.856313: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 26959440 exceeds 10% of free system memory.


...

3511/3511 [==============================] - 5s 2ms/step


(run_model pid=10462, ip=192.168.0.12) 2023-01-07 23:54:25.300690: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 33699300 exceeds 10% of free system memory.


13432/17552 [=====================>........] - ETA: 5s


(run_model pid=10753, ip=192.168.0.12) 2023-01-07 23:54:42.305518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 33699300 exceeds 10% of free system memory.


17552/17552 [==============================] - 19s 1ms/step
(561655, 20) (561655, 5)


In [7]:
'''def run_model_after_cncat(new_x_train , y_train, new_x_test , y_test):
    #individual classifier
    input_dim=new_x_train.shape[1]
    num_classes=len(set(y_train))

    model1=create_model()
    #hist = model1.fit(x_train , y_train , epochs=20)
    #hist = model1.fit(x_train , y_train , epochs=20, batch_size=5, verbose=0)
    print(new_x_train.shape)
    hist = model1.fit(new_x_train , y_train , epochs=num_epochs, batch_size=batch_size, verbose=0)
    predict=model1.predict(new_x_test)
    _, accuracy = model1.evaluate(new_x_test, y_test)
    print('Accuracy: %.2f' % (accuracy*100))

predictions2=run_model_after_cncat(new_x_train , y_train, new_x_test , y_test)'''


"def run_model_after_cncat(new_x_train , y_train, new_x_test , y_test):\n    #individual classifier\n    input_dim=new_x_train.shape[1]\n    num_classes=len(set(y_train))\n\n    model1=create_model()\n    #hist = model1.fit(x_train , y_train , epochs=20)\n    #hist = model1.fit(x_train , y_train , epochs=20, batch_size=5, verbose=0)\n    print(new_x_train.shape)\n    hist = model1.fit(new_x_train , y_train , epochs=num_epochs, batch_size=batch_size, verbose=0)\n    predict=model1.predict(new_x_test)\n    _, accuracy = model1.evaluate(new_x_test, y_test)\n    print('Accuracy: %.2f' % (accuracy*100))\n\npredictions2=run_model_after_cncat(new_x_train , y_train, new_x_test , y_test)"

In [8]:
input_dim=new_x_train.shape[1]

NN_clf1=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf1._estimator_type = "classifier"
NN_clf2=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf2._estimator_type = "classifier"
NN_clf3=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf3._estimator_type = "classifier"
NN_clf4=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf4._estimator_type = "classifier"
NN_clf5=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
NN_clf5._estimator_type = "classifier"


/tmp/ipykernel_40521/3367525961.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  NN_clf1=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
/tmp/ipykernel_40521/3367525961.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  NN_clf2=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
/tmp/ipykernel_40521/3367525961.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  NN_clf3=KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
/

In [9]:
intermediate = [('NN1', NN_clf1), ('NN2', NN_clf2), ('NN3', NN_clf3), ('NN4', NN_clf4),
             ('NN5', NN_clf5)]
intermediate


[('NN1', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa56c3f6710>),
 ('NN2', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa56c3f6ce0>),
 ('NN3', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa56c3f66b0>),
 ('NN4', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa56c3f65c0>),
 ('NN5', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fa56c3f6bf0>)]

In [10]:
# run after ad preictions of first level
t1=time.time()
@ray.remote(num_cpus=num_P)
def parallel_ensemble(algo,p_X,p_y):
    t_temp=time.time()
    y_predict = cross_val_predict(algo,p_X,p_y, cv=5) 
    #score = cross_val_score( algo,new_X,y,cv = 10,scoring = 'accuracy')
    #print("The accuracy score of {} is:".format(algo),score.mean(), "time: ", time.time()-t_temp)
    acc=accuracy_score(y_predict,y)
    prc=precision_score(y_predict,y, average='weighted')
    rec=recall_score(y_predict,y, average='weighted')
    f1=f1_score(y_predict,y, average='weighted')


    results = 'DL:%d' %(i) + ', Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
    results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', T:%.2f ' % (time.time()-t_temp) + ', PC: ' + PC
    results += ', Num Processors: ' + str(num_P) + ', Eb:%d ' %(num_epochs) + ', Bs:%d ' %(batch_size) + ', Date:' + Time_Now + '\n'
    
    return results

clf = [NN_clf1,NN_clf2,NN_clf3,NN_clf4, NN_clf5]
outputs_after=[parallel_ensemble.remote(algo, new_X,y) for algo in clf]

    
#collect the results of classification after get new features from the bae classifier.
path2=path + "results/Res_Singl_Parallel_DL_after.txt" # the name of the file
results_after='Dataset:' + dataset + '=============================================\n'

for i in range(5):
    results_after+=ray.get(outputs_after[i])
    
all_time_after=time.time()-t1

results_after+='Done...in time:' + str(round(all_time_after)) + '\n'
write_results(results_after, path2)

(parallel_ensemble pid=10886, ip=192.168.0.12) 2023-01-07 23:55:32.553958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(parallel_ensemble pid=10886, ip=192.168.0.12) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(parallel_ensemble pid=10886, ip=192.168.0.12) 2023-01-07 23:55:36.305147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(parallel_ensemble pid=10886, ip=192.168.0.12) 2023-01-07 23:55:36.305199: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(raylet, ip=192.168.0.12) [2023-01-07 23:55:40,801 E 103

1703/3511 [=============>................] - ETA: 2s12)[0


(parallel_ensemble pid=10886, ip=192.168.0.12) 2023-01-08 00:05:02.570481: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 35945920 exceeds 10% of free system memory.


3511/3511 [==============================] - 5s 1ms/step[0


(parallel_ensemble pid=11125, ip=192.168.0.12) 2023-01-08 00:05:19.289297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(parallel_ensemble pid=11125, ip=192.168.0.12) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(parallel_ensemble pid=11125, ip=192.168.0.12) 2023-01-08 00:05:22.954147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(parallel_ensemble pid=11125, ip=192.168.0.12) 2023-01-08 00:05:22.954192: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(parallel_ensemble pid=11125, ip=192.168.0.12) 2023-01-0

...

3511/3511 [==============================] - 4s 1ms/step


(raylet, ip=192.168.0.12) [2023-01-08 00:41:40,972 E 10357 10357] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 1d70fc76e3e3c0f69400e80f49ed89e148688966e12d60a006d75b29, IP: 192.168.1.3) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.3`
(raylet, ip=192.168.0.12) 
(raylet, ip=192.168.0.12) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(parallel_ensemble pid=11389, ip=192.168.0.12) 2023-01-08 00:41:41.3

3511/3511 [==============================] - 6s 2ms/step[0


(raylet, ip=192.168.0.12) [2023-01-08 00:58:40,994 E 10357 10357] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 1d70fc76e3e3c0f69400e80f49ed89e148688966e12d60a006d75b29, IP: 192.168.1.3) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.3`
(raylet, ip=192.168.0.12) 
(raylet, ip=192.168.0.12) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(parallel_ensemble pid=11516, ip=192.168.0.12) 2023-01-08 00:58:43.6

3511/3511 [==============================] - 3s 934us/step


In [ ]:
#Ensemble stacked results
t2=time.time()
#kfold = KFold(n_splits=10, shuffle=True)
clf = StackingClassifier(estimators=intermediate, final_estimator=LogisticRegression()) #MLPClassifier()

clf.fit(new_x_train , y_train)
#score = cross_val_score(stack_model,new_X,y,cv = 10,scoring = 'accuracy')
#score
y_predict=clf.predict(new_x_test)
acc=accuracy_score(y_predict,y_test)
prc=precision_score(y_predict,y_test, average='weighted')
rec=recall_score(y_predict,y_test, average='weighted')
f1=f1_score(y_predict,y_test, average='weighted')

Ense_results ='Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
Ense_results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', PC: ' + PC
Ense_results += ', Num Processors: ' + str(num_P) +  ', Eb:%d ' %(num_epochs) + ', Bs:%d ' %(batch_size) + 'Dataset: ' + dataset  + ', Date:' + Time_Now + '\n'
  
Ense_results+='Done...in time:' + str(round(time.time()-t2)) + '\n'

path2= path + "results/Res_Parallel_Ensemble_DL.txt" # the name of the file
write_results(Ense_results, path2)

print("Done...in time: ", time.time()-t2)

/home/alandoli/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/alandoli/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
2023-01-08 01:38:57.241431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-08 01:38:57.242021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: ca

3160/3160 [==============================] - 4s 1ms/step


(raylet) [2023-01-08 02:56:20,277 E 32627 32627] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 5a654be6b0c11a4fb60832b3623982da0e6deca6db144f723384f2e0, IP: 192.168.1.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


3160/3160 [==============================] - 4s 1ms/step


(raylet) [2023-01-08 03:56:20,419 E 32627 32627] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 5a654be6b0c11a4fb60832b3623982da0e6deca6db144f723384f2e0, IP: 192.168.1.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [ ]:
print("Stacking model score: %.3f" % clf.score(new_x_test , y_test))

In [ ]:
ray.shutdown()